In [2]:
import os
import pandas as pd
import xlrd
from pathlib import Path
import openpyxl
from openpyxl import load_workbook
import numpy as np
import re
import ast
from xls2xlsx import XLS2XLSX

In [2]:

    ##Getting the 1045 data
xlsx_file = Path('SimData', '/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster/Alvarado Hospital/106370652_CDM_All_2020.xlsx')
wb_obj = openpyxl.load_workbook(xlsx_file) 

In [3]:
#Function to extract 1045 sheet name from xlsx file only

def find1045_xlsx(filepath):
    #read file
    xlsx_file = Path(filepath)
    wb_obj = openpyxl.load_workbook(xlsx_file)

    #get sheet names
    sheetnames=wb_obj.sheetnames

    #Create a keywords list to identify ABS 1045 forms
    keywords= ["common", "procedures", "1045", "form"]

    #Search for the sheet by keyword
    for name in sheetnames:
        wholename=name.lower()
        wordsInName=wholename.split()
        match =  any(item in wordsInName for item in keywords)
        if match==True:
            sheetname=(wb_obj[name])
            return(sheetname)
            break


In [4]:
#Function to extract all dataframes from a single xlsx file
def getdata_xlsx(file):
    dflist=[]
    coordlist=[]
    sheet=find1045_xlsx(file)
    alphabet=["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
    found=False
    keywords1 = ["evaluation", "management", "services"]
    keywords2 = ["laboratory", "pathology", "services"]
    keywords3= ["radiology", "services"]
    keywords4 = ["medicine", "services"]
    keywords5 = ["surgery", "services"]
    keywords6 = ["other", "common", "outpatient", "procedures"]
    keywords7 = ["count", "of", "reported", "procedures",]
    keywordslists=[keywords1,keywords2,keywords3,keywords4,keywords5,keywords6,keywords7]
    for wordlist in keywordslists:
        for row in sheet.iter_rows():
                for cell in row:
                    heading = str(cell.value).lower()
                    wordsInHeading=heading.split()
                    match =  all(item in wordsInHeading for item in wordlist)
                    if match==True:
                        coord1=cell.coordinate
                        coordlist.append(coord1)
                        break
    for coordinate in coordlist[0:-1]:
        splitcoord=re.split('(\d+)', coordinate)
        nextbreak=coordlist[int(coordlist.index(coordinate)+1)]
        splitbreak=re.split('(\d+)', nextbreak)
        lastcell=f"{int(splitbreak[1])-1}"
        coord2=f"C{lastcell}"

        cellgroup=[coordinate,coord2]
        df=pd.DataFrame(sheet[f"{cellgroup[0]}:{cellgroup[1]}"])



        dflist.append(df)

    return(dflist)

  
    

In [5]:
#Function to convert cell object values to plaintext values
def convertvalue(x):
    return x.value

In [6]:
#Convert all dataframes from a single sheet into plaintext
sheet=find1045_xlsx(xlsx_file)
masterlist=[]
rawlist=getdata_xlsx(xlsx_file)
for i in range(len(rawlist)):
    df=rawlist[i].applymap(convertvalue)
            #add the hospital name as a variable
    masterlist.append(df)

In [7]:
#Get the column names from the dataframe
for df in masterlist:
    if len(df)>0:
        df.columns = df.iloc[0]
        df["Service Type"]=df.iloc[0,0]
        df.rename(columns={ df.columns[0]: "Service" }, inplace = True)
        df=df.drop(df.index[0], inplace=True)




In [8]:
#add the hospital name as a variable in every df in the converted list
for i in range(len(masterlist)):
        raw_name=sheet["A1"].value
        hos_name=raw_name.replace('Hospital Name: ', '')
        masterlist[i]["Hospital"]=hos_name


In [9]:
masterlist[0]

,Service,2019 CPT Code,Average Charge,Service Type,Hospital
1,"Emergency Room Visit, Level 2 (low to moderate...",99282,746.55,Evaluation & Management Services (CPT Codes 99...,Alvarado Hospital
2,"Emergency Room Visit, Level 3 (moderate severity)",99283,1051.05,Evaluation & Management Services (CPT Codes 99...,Alvarado Hospital
3,"Emergency Room Visit, Level 4 (high severity)",99284,1581.3,Evaluation & Management Services (CPT Codes 99...,Alvarado Hospital
4,"Outpatient Visit, established patient, 15 minutes",99213,None,Evaluation & Management Services (CPT Codes 99...,Alvarado Hospital


In [10]:
#get the names of all folders in the data directory
os.chdir("..")
os.chdir("data")
os.chdir("2020_CDM_chargemaster")



In [11]:
#Function to extract 1045 sheet name

def find1045(filepath):
    #read file
    if filepath[-5:] == ".xlsx":
        excel_file = Path(filepath)
        wb_obj = openpyxl.load_workbook(excel_file)

    #get sheet names
    sheetnames=wb_obj.sheetnames

    #Create a keywords list to identify ABS 1045 forms
    keywords= ["common", "procedures", "1045", "form"]

    #Search for the sheet by keyword
    for name in sheetnames:
        wholename=name.lower()
        wordsInName=wholename.split()
        match =  any(item in wordsInName for item in keywords)
        if match==True:
            sheetname=(wb_obj[name])
            return(sheetname)
            break


In [12]:
#get the names of all files from each directory, then test if each file has a 1045 in it.
dirlist = list(os. listdir())
for i in range(len(dirlist)):
    os.chdir(dirlist[i])
    for root, dirs, files in os.walk("."):
        for filename in files:
            print(filename)
            print (find1045_xlsx(filename))
    os.chdir("..")

106390923_CDM_All_2020.xlsx
<Worksheet "Common OP Procedures">
106104023_CDM_All_2020.xls


InvalidFileException: openpyxl does not support the old .xls file format, please use xlrd to read this file, or convert it to the more recent .xlsx file format.

In [ ]:
filename="test.xlsx"

In [ ]:
filename[-5:]

In [15]:
os.chdir("data")

FileNotFoundError: [Errno 2] No such file or directory: 'data'

In [16]:
os.getcwd()

'/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster/San Joaquin Valley Rehab Center'

In [17]:
os.chdir("..")

In [18]:
os.chdir("2020_CDM_chargemaster")

FileNotFoundError: [Errno 2] No such file or directory: '2020_CDM_chargemaster'

In [3]:

    ##Getting the 1045 data
xls_file = Path("/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster/Emanate Health Foothill Presbyterian Hospital/106190298_Common25_2020.xls")

In [20]:
book = xlrd.open_workbook(xls_file)

In [21]:
book = xlrd.open_workbook(xls_file)

print("The number of worksheets is {0}".format(book.nsheets))
print("Worksheet name(s): {0}".format(book.sheet_names()))
sh = book.sheet_by_index(0)
print("{0} {1} {2}".format(sh.name, sh.nrows, sh.ncols))
print("Cell D30 is {0}".format(sh.cell_value(rowx=29, colx=3)))
for rx in range(sh.nrows):
    print(sh.row(rx))

The number of worksheets is 2
Worksheet name(s): ['Top 50 List', 'AB 1045 Form']
Top 50 List 51 3


IndexError: list index out of range

In [27]:
#Function to extract 1045 sheet from xls file (not name, whole sheet)

def find1045_xls(filepath):
    #read file
    xls_file = Path(filepath)
    book = xlrd.open_workbook(xls_file)

    #get sheet names
    sheetnames=("{0}".format(book.sheet_names()))
    sheetnames= ast.literal_eval(sheetnames)


#Create a keywords list to identify ABS 1045 forms
    keywords= ["common", "procedures", "1045", "form"]

#Search for the sheet by keyword
    for name in sheetnames:
        wholename=name.lower()
        wordsInName=wholename.split()
        match =  any(item in wordsInName for item in keywords)
        if match==True:
            cur_sheet = book.sheet_by_name(name)
            return(cur_sheet)
            break


In [35]:
x2x = XLS2XLSX(xls_file)


AttributeError: 'PosixPath' object has no attribute 'read'

In [36]:
from xls2xlsx import XLS2XLSX
x2x = XLS2XLSX(xls_file)
wb = x2x.to_xlsx()

AttributeError: 'PosixPath' object has no attribute 'read'

In [ ]:
884

>>> import ast
>>> x = '[ "A","B","C" , " D"]'
>>> x = ast.literal_eval(x)
>>> x
['A', 'B', 'C', ' D']
>>> x = [n.strip() for n in x]
>>> x
['A', 'B', 'C', 'D']

In [ ]:

import xlrd
from openpyxl.workbook import Workbook as openpyxlWorkbook

# content is a string containing the file. For example the result of an http.request(url).
# You can also use a filepath by calling "xlrd.open_workbook(filepath)".

xlsBook = xlrd.open_workbook(file_contents=content)
workbook = openpyxlWorkbook()

for i in xrange(0, xlsBook.nsheets):
    xlsSheet = xlsBook.sheet_by_index(i)
    sheet = workbook.active if i == 0 else workbook.create_sheet()
    sheet.title = xlsSheet.name

    for row in xrange(0, xlsSheet.nrows):
        for col in xrange(0, xlsSheet.ncols):
            sheet.cell(row=row + 1, column=col + 1).value = xlsSheet.cell_value(row, col)

# The new xlsx file is in "workbook", without iterators (iter_rows).
# For iteration, use "for row in worksheet.rows:".
# For range iteration, use "for row in worksheet.range("{}:{}".format(startCell, endCell)):".

In [64]:
#Iterate through files and convert all xls files to xlsx, removing the original file
def convert(filename):
    p.save_book_as(file_name=f"{filename}",
                dest_file_name=f"{filename[:-4]}.xlsx" )

In [75]:
import pyexcel as p
data=p.get_records(file_name=xls_file)
p.save_as(array=data, dest_file_name="example.xls")

In [74]:
os.getcwd()

'/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster'

In [63]:
def cleanse_func(v):
     v = v.replace("&nbsp;", "")
     v = v.rstrip().strip()
     return v


In [60]:
from pyexcel_xls import get_data
data = get_data(xls_file)


In [54]:
cwd=os.getcwd()

In [56]:
os.getcwd()

'/home/ygael/Projects/healthcare/data/2019_CDM_chargemaster'

In [52]:
os.chdir("..")

In [53]:
cwd

'/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster'

In [15]:
os.chdir("data")

In [17]:
os.chdir("2020_CDM_chargemaster")

In [81]:
#A function to convert all files in subfolders starting from the absolute filepath of the year's chargemaster folder

def convertfrom(directory):
    #Use absolute path to move to the directory containing all hospital folders
    os.chdir(directory)
    #save current working directory to a variable
    cwd=os.getcwd()
    #Convert all files to xlsx format
    dirlist = list(os. listdir())
    for i in range(len(dirlist)):
        os.chdir(dirlist[i])
        for root, dirs, files in os.walk("."):
            for filename in files:
                if filename[-3:]=="xls":
                    try:
                        convert(filename)
                        os.remove(filename)
                    except:
                        print(filename)
                else:
                    continue
        os.chdir("..")
    os.chdir(cwd)

In [82]:
convertfrom("/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster")

106364188_CDM_2020.xls
106364188_CDM_2020-Cloud.xls
106332172_CDM_2020.xls
106332172_CDM_2020-Cloud.xls
106201281_PCT_CHG_2020.xls


In [70]:
from import_export import resources
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE

class CleanModelResource(resources.ModelResource):
  def export_field(self, field, obj):
    v = super(CleanModelResource, self).export_field(field, obj)
    if type(v) == str:
      v = ILLEGAL_CHARACTERS_RE.sub('', v)
    return v

In [5]:
xls_file = "/home/ygael/Projects/healthcare/data/2019_CDM_chargemaster/Kindred Hospital - San Francisco Bay Area/106010887_PCT_CHG_2019.xls"

data=pd.read_excel(xls_file)
data = data.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)
data.to_excel

In [11]:
os.getcwd()

'/home/ygael/Projects/healthcare/code'

In [13]:
import re
import xlrd

def removeSpecialCharacters(cellValue):
    text=re.sub(r"[\r\n\t\x07\x0b]", "", cellValue)
    return text

In [16]:
book = xlrd.open_workbook(xls_file)


#character = table.Cell(Row = 1, Column = 1).Range.Text
#character = removeSpecialCharacters(character)

In [17]:
for item in book:
    print(item)

TypeError: 'Book' object is not iterable

In [25]:
def find1045_xls(filepath):
    #read file
    xls_file = Path(filepath)
    book = xlrd.open_workbook(xls_file)

    #get sheet names
    sheetnames=("{0}".format(book.sheet_names()))
    sheetnames= ast.literal_eval(sheetnames)


#Create a keywords list to identify ABS 1045 forms
    keywords= ["common", "procedures", "1045", "form"]

    cur_sheet = book.sheet_by_name(sheetnames[0])
    return(cur_sheet)

In [26]:
sheet=find1045_xls(xls_file)

In [29]:
for item in sheet:
    print(item)

TypeError: 'Sheet' object is not iterable

In [33]:
row = sheet.row(10)

In [53]:
num_cols = sheet.ncols
for row_idx in range(0, sheet.nrows):    # Iterate through rows
    print ('Row: %s' % row_idx)   # Print row number
    for col_idx in range(0, num_cols):  # Iterate through columns
        cell_obj = sheet.cell(row_idx, col_idx)  # Get cell object by row, col
        removeSpecialCharacters([item for item in cell_obj])


Row: 0


TypeError: 'Cell' object is not iterable

In [36]:
num_cols

10

In [40]:

#importing pandas as pd
import pandas as pd
  
# Read and store content
# of an excel file 
read_file = pd.read_excel (xls_file)
  
# Write the dataframe object
# into csv file
read_file.to_csv ("Test.csv", 
                  index = None,
                  header=True)
    
# read csv file and convert 
# into a dataframe object
df = pd.DataFrame(pd.read_csv("Test.csv"))
  
# show the dataframe
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Max of STD Amount,STD EFFEff Date,NaN,NaN,NaN,2018,Utilzation,Annualized,Estimated,Price
2,Charge,2018,2019.00,Grand Total,NaN,Increase,2019 QTR01,Projection,Gross Rev Impact,Differential
3,10800018,3911,4184.77,4184.77,NaN,0.07000000000000012,5280,21120,5782022.40000001,273.77000000000044
4,10800300,4770,5103.90,5103.9,NaN,0.06999999999999992,1453,5812,1940626.799999998,333.89999999999964
...,...,...,...,...,...,...,...,...,...,...
310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13079118.000000013,Est Revenue Increase
313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,233750033,2018 Total Gross Revenue


In [41]:
df.to_excel("test.xlsx")

In [47]:
sheet.decode('utf-8',errors='ignore')

AttributeError: 'Sheet' object has no attribute 'decode'

In [48]:
import re

def removeSpecialCharacters(cellValue):
    text=re.sub(r"[\r\n\t\x07\x0b]", "", cellValue)
    return text

In [50]:
for row in sheet:
    print(row)

TypeError: 'Sheet' object is not iterable

In [76]:
def convertxlsm(filename):
    p.save_book_as(file_name=f"{filename}",
                dest_file_name=f"{filename[:-4]}.xlsm" )

In [84]:
database = MySQLdb.connect (host="localhost", user = "User_name", passwd = "Password", db = "Database_name")

NameError: name 'MySQLdb' is not defined

In [90]:
import xlrd
import MySQLdb
import re
# Open the workbook and define the worksheet
book= xlrd.open_workbook(xls_file)
sheet = book.sheet_by_name("Sheet_Name")

# Establish a MySQL connection
database = MySQLdb.connect (host="localhost", user = "User_name", passwd = "Password", db = "Database_name")

# Get the cursor, which is used to traverse the database, line by line
cursor = database.cursor()

# Create the INSERT INTO sql query


query = """INSERT INTO Table_name  VALUES (%s,%s)"""      

# Create a For loop to iterate through each row in the XLS file, 
for r in range(1, sheet.nrows):
  
  Column_name1=re.sub("\d+","",sheet.cell(r,0).value).strip(")")   #removes numeric charcters and special charcters ; "()" here
  Column_name1=Column_name1[:-1]
  Column_name2=re.sub("\d+","",sheet.cell(r,1).value).strip(")")
  Column_name2=Column_name2[:-1]
  
  # Assign values from each row
  values =(Column_name1,Column_name2)
  # Execute sql Query
  try:
      
      cursor.execute(query,values)
  except:
      pass



# Close the cursor
cursor.close()

# Commit the transaction
database.commit()

# Close the database connection
database.close()



XLRDError: No sheet named <'Sheet_Name'>